# Sonar - Decentralized Model Training Simulation (local)

DISCLAIMER: This is a proof-of-concept implementation. It does not represent a remotely product ready implementation or follow proper conventions for security, convenience, or scalability. It is part of a broader proof-of-concept demonstrating the vision of the OpenMined project, its major moving parts, and how they might work together.


# Getting Started: Installation

##### Step 1: install IPFS

- https://ipfs.io/docs/install/

##### Step 2: Turn on IPFS Daemon
Execute on command line:
> ipfs daemon

##### Step 3: Install Ethereum testrpc

- https://github.com/ethereumjs/testrpc

##### Step 4: Turn on testrpc with 1000 initialized accounts (each with some money)
Execute on command line:
> testrpc -a 1000

##### Step 5: install openmined/sonar and all dependencies (truffle)

##### Step 6: Locally Deploy Smart Contracts in openmined/sonar
From the OpenMined/Sonar repository root run
> truffle compile
> truffle migrate

you should see something like this when you run migrate:
```
Using network 'development'.

Running migration: 1_initial_migration.js
  Deploying Migrations...
  Migrations: 0xf06039885460a42dcc8db5b285bb925c55fbaeae
Saving successful migration to network...
Saving artifacts...
Running migration: 2_deploy_contracts.js
  Deploying ConvertLib...
  ConvertLib: 0x6cc86f0a80180a491f66687243376fde45459436
  Deploying ModelRepository...
  ModelRepository: 0xe26d32efe1c573c9f81d68aa823dcf5ff3356946
  Linking ConvertLib to MetaCoin
  Deploying MetaCoin...
  MetaCoin: 0x6d3692bb28afa0eb37d364c4a5278807801a95c5
```

The address after 'ModelRepository' is something you'll need to copy paste into the code
below when you initialize the "ModelRepository" object. In this case the address to be
copy pasted is `0xe26d32efe1c573c9f81d68aa823dcf5ff3356946`.

##### Step 7: execute the following code

# The Simulation: Diabetes Prediction

In this example, a diabetes research center (Cure Diabetes Inc) wants to train a model to try to predict the progression of diabetes based on several indicators. They have collected a small sample (42 patients) of data but it's not enough to train a model. So, they intend to offer up a bounty of $5,000 to the OpenMined commmunity to train a high quality model.

As it turns out, there are 400 diabetics in the network who are candidates for the model (are collecting the relevant fields). In this simulation, we're going to faciliate the training of Cure Diabetes Inc incentivizing these 400 anonymous contributors to train the model using the Ethereum blockchain.

Note, in this simulation we're only going to use the sonar and syft packages (and everything is going to be deployed locally on a test blockchain). Future simulations will incorporate mine and capsule for greater anonymity and automation.

### Imports and Convenience Functions

In [1]:
import warnings
import numpy as np
import phe as paillier
from sonar.contracts import ModelRepository,Model
from syft.he.Paillier import KeyPair
from syft.nn.linear import LinearClassifier
import numpy as np
from sklearn.datasets import load_diabetes

def get_balance(account):
    return repo.web3.fromWei(repo.web3.eth.getBalance(account),'ether')

warnings.filterwarnings('ignore')

### Setting up the Experiment

In [5]:
# for the purpose of the simulation, we're going to split our dataset up amongst
# the relevant simulated users

diabetes = load_diabetes()
y = diabetes.target
X = diabetes.data

validation = (X[0:5],y[0:5])
anonymous_diabetes_users = (X[6:],y[6:])

# we're also going to initialize the model trainer smart contract, which in the
# real world would already be on the blockchain (managing other contracts) before
# the simulation begins

# ATTENTION: copy paste the correct address (NOT THE DEFAULT SEEN HERE) from truffle migrate output.
repo = ModelRepository('0xb0f99be3d5c858efaabe19bcc54405f3858d48bc', ipfs_host='localhost', web3_host='localhost') # blockchain hosted model repository

No account submitted... using default[2]
Connected to OpenMined ModelRepository:0xb0f99be3d5c858efaabe19bcc54405f3858d48bc


In [6]:


# we're going to set aside 10 accounts for our 42 patients
# Let's go ahead and pair each data point with each patient's 
# address so that we know we don't get them confused
patient_addresses = repo.web3.eth.accounts[1:10]
anonymous_diabetics = list(zip(patient_addresses,
                               anonymous_diabetes_users[0],
                               anonymous_diabetes_users[1]))

# we're going to set aside 1 account for Cure Diabetes Inc
cure_diabetes_inc = repo.web3.eth.accounts[1]

## Step 1: Cure Diabetes Inc Initializes a Model and Provides a Bounty

In [7]:
pubkey,prikey = KeyPair().generate(n_length=1024)
diabetes_classifier = LinearClassifier(desc="DiabetesClassifier",n_inputs=10,n_labels=1)
initial_error = diabetes_classifier.evaluate(validation[0],validation[1])
diabetes_classifier.encrypt(pubkey)

diabetes_model = Model(owner=cure_diabetes_inc,
                       syft_obj = diabetes_classifier,
                       bounty = 1,
                       initial_error = initial_error,
                       target_error = 10000
                      )
model_id = repo.submit_model(diabetes_model)

## Step 2: An Anonymous Patient Downloads the Model and Improves It

In [8]:
model_id

0

In [9]:
model = repo[model_id]

In [10]:
diabetic_address,input_data,target_data = anonymous_diabetics[0]

In [11]:
repo[model_id].submit_gradient(diabetic_address,input_data,target_data)

## Step 3: Cure Diabetes Inc. Evaluates the Gradient 

In [12]:
repo[model_id]

Desc:DiabetesClassifier
Owner:0xeddf96ff03e9d7883263c73ad75d1fd68583a398
Bounty:1
Initial Error:21793600
Best Error:None
Target Error:10000
Model ID:0
Num Grads:1

In [13]:
old_balance = get_balance(diabetic_address)
print(old_balance)

98.999999999999645746


In [14]:
new_error = repo[model_id].evaluate_gradient(cure_diabetes_inc,repo[model_id][0],prikey,pubkey,validation[0],validation[1])

In [15]:
new_error

21739961

In [16]:
new_balance = get_balance(diabetic_address)
incentive = new_balance - old_balance
print(incentive)

0.002461227149139725


## Step 4: Rinse and Repeat

In [19]:
model

Desc:DiabetesClassifier
Owner:0xeddf96ff03e9d7883263c73ad75d1fd68583a398
Bounty:1
Initial Error:21793600
Best Error:None
Target Error:10000
Model ID:0
Num Grads:7

In [18]:
for i,(addr, input, target) in enumerate(anonymous_diabetics):
    try:
        
        model = repo[model_id]
        
        # patient is doing this
        model.submit_gradient(addr,input,target)
        
        # Cure Diabetes Inc does this
        old_balance = get_balance(addr)
        new_error = model.evaluate_gradient(cure_diabetes_inc,model[i+1],prikey,pubkey,validation[0],validation[1],alpha=2)
        print("new error = "+str(new_error))
        incentive = round(get_balance(addr) - old_balance,5)
        print("incentive = "+str(incentive))
    except:
        "Connection Reset"

new error = 21637218
incentive = 0.00473
new error = 21749031
incentive = 0.00000
new error = 21594788
incentive = 0.00196
